In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline

In [2]:
#PART 2 OF ASSIGMENT 4, Build and train deeper CNNs with tf.nn.dropout
# • Add more Conv layers
# • Add dropout function after each pooling layer
# best performance achieved is 0.98
# only visualization part is not done

print ("Download and Extract MNIST dataset")
mnist = input_data.read_data_sets('data/', one_hot=True)
print (" tpye of 'mnist' is ", type(mnist))
print (" number of trian data is %d" % (mnist.train.num_examples))
print (" number of test data is %d" % (mnist.test.num_examples))

# learning arameters
learning_rate = 0.01
noOfIterations = 100000
batch_size = 100
display_step = 10

# net parameters
n_input_width = 28 # MNIST images are 28 pixels * 28 pixels = 784
n_input_height = 28

n_output = 10 # classes (0-9 digits)
dropout = 0.9 # dropout, prob. to keep units in the process

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input_width * n_input_height])
y = tf.placeholder(tf.float32, [None, n_output])
keep_prob = tf.placeholder(tf.float32) #dropout in feed_dict

# helper function to do conv2d, bias adding and relu activation together
def conv2d(input, weights, biases):
    # Convolution
    conv = tf.nn.conv2d(input, weights, strides=[1, 1, 1, 1], padding='SAME')
    # Add-bias
    bias_addition = tf.nn.bias_add(conv, biases)
    # Pass ReLu
    relu_result = tf.nn.relu(bias_addition)
    return relu_result

# helper function to do "k" max-pooling
def max_pool(input, k):
    return tf.nn.max_pool(input, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

# helper function to apply dropout to an intended layer
def apply_dropout(layer, dropout):
    return tf.nn.dropout(layer, dropout)

# helper fucntion to costruct network model
def construct_model(X, weights, biases, dropout):
    
    # reshape input image
    input_r = tf.reshape(X, shape=[-1, 28, 28, 1])

    # convolution layer 1
    conv1 = conv2d(input_r, weights['wc1'], biases['bc1'])
    # max pooling with k = 2
    pool1 = max_pool(conv1, k=2)
    # apply dropout after pooling layer
    pool1_drop = apply_dropout(pool1, dropout)

    # convolution layer 2
    conv2 = conv2d(pool1_drop, weights['wc2'], biases['bc2'])
    # max pooling with k = 2
    pool2 = max_pool(conv2, k=2)
    # apply dropout after pooling layer
    pool2_drop = apply_dropout(pool2, dropout)

    # fully connected layer
    dense = tf.reshape(pool2_drop, [-1, weights['wf1'].get_shape().as_list()[0]]) # vectorize pool2_drop
    relu = tf.nn.relu(tf.add(tf.matmul(dense, weights['wf1']), biases['bf1'])) # apply relu
    relu_drop = apply_dropout(relu, dropout) # apply dropout

    # output - class prediction
    out = tf.add(tf.matmul(relu_drop, weights['out']), biases['out'])
    return out

# define weights and biases arrays
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])), # 5x5 convolution, 1 input channel, 32 filters
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])), # 5x5 convolution, 32 input channel, 64 filters
    'wf1': tf.Variable(tf.random_normal([7 * 7 * 64, 1024])), # fully connected, 7 * 7 * 64 inputs, 1024 outputs
    'out': tf.Variable(tf.random_normal([1024, n_output])) # 1024 inputs, 10 class outputs
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bf1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_output]))
}

# construct model
pred = construct_model(x, weights, biases, keep_prob)

# define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

# test model for evaluation
predictions = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
# calculate accuracy
accuracy = tf.reduce_mean(tf.cast(predictions, "float"))

# init the variables
init = tf.initialize_all_variables()

# open session
with tf.Session() as sess:
    sess.run(init)
    
    # define number of steps to print training accuracy, and cost of the batch
    iter = 1
    
    # training, stop when maximum noOfIterations reached
    while iter * batch_size < noOfIterations:
        
        # get bacth instances
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            
        # fit training using batch
        sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys, keep_prob: dropout})
            
        if iter % display_step == 0:
            
            # calculate accuracy in the batch
            acc = sess.run(accuracy, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
            
            # calculate cost function in the batch
            loss = sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
            
            print "Iteration " + str(iter * batch_size) + ", batch cost = " + "{:.5f}".format(loss) + ", training acc = " + "{:.5f}".format(acc)
        
        iter += 1

    print "\nResults:\n"

    # calculate test accuracy
    print "Test Accuracy:", sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.})
    
    # ONLY VISUALIZATION PART IS NOT DONE
    #for i in range(64):
        #plt.matshow(pool[0, :, :, i], cmap=plt.get_cmap('gray'))
        #plt.title(str(i) + "th pool")
        #plt.colorbar()
        #plt.show()
    
    writer = tf.train.SummaryWriter('/tmp/tf_logs/cnn_model',sess.graph)

Download and Extract MNIST dataset
Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
(" tpye of 'mnist' is ", <class 'tensorflow.contrib.learn.python.learn.datasets.mnist.DataSets'>)
 number of trian data is 55000
 number of test data is 10000
Iteration 1000, batch cost = 16681.36719, training acc = 0.74000
Iteration 2000, batch cost = 2922.60229, training acc = 0.72000
Iteration 3000, batch cost = 1058.01392, training acc = 0.80000
Iteration 4000, batch cost = 1017.96234, training acc = 0.78000
Iteration 5000, batch cost = 392.43359, training acc = 0.86000
Iteration 6000, batch cost = 501.31686, training acc = 0.88000
Iteration 7000, batch cost = 375.50790, training acc = 0.87000
Iteration 8000, batch cost = 312.42477, training acc = 0.79000
Iteration 9000, batch cost = 450.63040, training acc = 0.81000
Iteration 10000, batch cost = 320.63202, training acc = 0.83000
